In [ ]:
! pip install Flask


In [ ]:
! pip install import_ipynb

In [ ]:
! pip install pymysql

In [ ]:
! pip install WTForms

In [ ]:
! pip install mysqlclient

In [ ]:
! pip install flask-mysqldb


In [ ]:
! pip install flask-login

In [ ]:
! pip freeze

In [9]:
from flask import Flask, request, render_template, redirect, url_for, jsonify
import import_ipynb
import forms, hashlib 
from flask_mysqldb import MySQL

In [ ]:
# intilize flask application
app = Flask(__name__)

In [11]:
app.config["SECRET_KEY"]="Hola como etas muy bein y tu?"
app.config['MYSQL_HOST']="localhost"
app.config["MYSQL_USER"]="root"
app.config["MYSQL_PASSWORD"]=""
app.config["MYSQL_DB"]="schoolapp"

In [12]:
mysql = MySQL(app)

In [ ]:
@app.route("/")
def home():
    return redirect(url_for("login"))

In [ ]:
@app.route("/login",methods=["GET","POST"])
def login():
    if request.method=="POST":
        login_form=forms.LoginForm(request.form)
        if login_form.validate():
            return jsonify(login_form.data)
        else :
            return jsonify({"errors": login_form.errors}),400
    else :
        return render_template("auth/user.html")

In [15]:

@app.route("/register",methods=["GET","POST"])
def register():
    if request.method=="POST":
        print(request.form)
        register_form=forms.RegistrationForm(request.form)
        print(register_form.data)
        if register_form.validate():
            # return jsonify(register_form.data)
            user_data= request.form
            first_name=user_data["first_name"]
            last_name=user_data["last_name"]
            middle_name=user_data["middle_name"]
            email=user_data["email"]
            password=user_data["password"]
            phone_number=user_data["phone_number"]
            hash=password+app.secret_key
            hash=hashlib.sha1(hash.encode())
            password=hash.digest()
            cursor=mysql.connection.cursor()
            cursor.execute("select email from users where email=%s",(email,))

            account = cursor.fetchone()
            if account:
                return "account exist"
            else: 
                cursor.execute("insert into users ( `first_name`, `last_name`, `middle_name`, `email`, `password`, `gender`, `mobile_number`, `whatsapp_number`, `address`, `city`, `state`, `country`, `profile_picture`, `role`, `status`,)values(%s,%s,%s,%s,%s,%s.%s,%s,%s,%s,%s,%s,%s,%s,%s)",(first_name,last_name,middle_name,email,password,"",phone_number,"","","","","","users","active"))
                mysql.connection.commit()
                cursor.close()
                return "account doesnt exist"


        else :
            return jsonify({"errors" : register_form.errors }), 400
    else :
        return render_template("auth/registartion.html")



In [ ]:
if __name__== "__main__":
    app.run(debug=False)